# Part 1 - Completed together

How many movies are there in the `Movies` table?

In [1]:
select count(TitleID)
from imdb.Movies

(1 row affected)

Total execution time: 00:00:00.068

(No column name)
130405


How many movies are there in the `Movies` table?  Name the column containing the count `Total Movies`.

In [2]:
select count(TitleID) as [Number of Movies]
from imdb.Movies

(1 row affected)

Total execution time: 00:00:00.026

Number of Movies
130405


How many movies are there that were released in **2010**?  How many unique `Title` values are there for 2010?

In [5]:
WITH MoviesIn2010 ([Movies made in 2010])
as (
select count(TitleID) as NumMovies
from imdb.Movies
where YearReleased = 2010
),
Distinct2010Movies([Unique titles in 2010])
as (
select count(distinct Title) as NumMovies
from imdb.Movies
where YearReleased = 2010
)
select * 
from MoviesIn2010, Distinct2010Movies

(1 row affected)

Total execution time: 00:00:00.055

Movies made in 2010,Unique titles in 2010
4785,4749


How many `Principals` with a `Category` of _actress_ or _actor_ were there in the 2010 movie titled _Bittersweet_?  How many of them were unique?

In [5]:
select count(Principals.PersonID) as [Bittersweet Actors]
from imdb.Movies, imdb.Principals
where Movies.TitleID = Principals.TitleID
and IsActor = 1
and YearReleased = 2010
and Title = 'Bittersweet';

select count(distinct Principals.PersonID) as [Distinct Bittersweet Actors]
from imdb.Movies, imdb.Principals
where Movies.TitleID = Principals.TitleID
and IsActor = 1
and YearReleased = 2010
and Title = 'Bittersweet';

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.029

Bittersweet Actors
8


Distinct Bittersweet Actors
7


What was the **average running time** for movies for which one of the genres was _Drama_ and was released in the same year as the movie titled _Christopher Robin_?  We'll use a **virtual table** in the `FROM` clause to help us with this.

In [3]:
with CRM
as
(
    select * from imdb.Movies
    where Title = 'Christopher Robin'
)
select AVG(Movies.RunningTimeInMinutes) [AVG Running Time In Minutes] from CRM, imdb.Movies
join imdb.MovieGenres 
  on MovieGenres.TitleID = Movies.TitleID
where Movies.YearReleased = CRM.YearReleased
  and Genre = 'Drama'

Warning: Null value is eliminated by an aggregate or other SET operation.

(1 row affected)

Total execution time: 00:00:00.054

AVG Running Time In Minutes
99


To avoid the warning, we can issue the statement
````
set ansi_warnings off
````
before executing the query:

In [5]:
set ansi_warnings OFF;

with CRM
as
(
    select * from imdb.Movies
    where Title = 'Christopher Robin'
)
select AVG(Movies.RunningTimeInMinutes) [AVG Running Time In Minutes] from CRM, imdb.Movies
join imdb.MovieGenres 
  on MovieGenres.TitleID = Movies.TitleID
where Movies.YearReleased = CRM.YearReleased
  and Genre = 'Drama'

(1 row affected)

Total execution time: 00:00:00.090

AVG Running Time In Minutes
99


Note that we only need to turn `ansi_warnings` off once.  It will stay in effect until we turn it back on, or we establish a new connection to the database server.

Finally, let's find which dramas were **longer** than the average length of movies made in the same year as _Christopher Robin_.

This can be done with a **subquery**, which we haven't discussed yet. But we can also do it using a **join**, with an **inequality** instead of an **equality** in the `on` portion of the `join`.

We'll use 3 different virtual tables to help us find the solution..

In [ ]:
with CR(YearReleased) as (
    select YearReleased
    from imdb.Movies
    where Title = 'Christopher Robin'
),
MoviesMadeSameYearAsCR(TitleID, Title, RunningTimeInMinutes) as (
  select TitleID, Title, RunningTimeInMinutes
  from imdb.Movies
  join CR 
    on CR.YearReleased = Movies.YearReleased
),
CRStatistics(AverageRunningTime) as ( 
    select avg(RunningTimeInMinutes)
    from MoviesMadeSameYearAsCR
    join imdb.MovieGenres
      on MoviesMadeSameYearAsCR.TitleID = MovieGenres.TitleID    
    where Genre = 'Drama'
) 
select Title, RunningTimeInMinutes, AverageRunningTime
from MoviesMadeSameYearAsCR
join CRStatistics 
  on MoviesMadeSameYearAsCR.RunningTimeInMinutes > CRStatistics.AverageRunningTime
order by RunningTimeInMinutes DESC  

# Part 2 - Complete on your own

How many different movies with the title _Titanic_ have there been, **other than** the one released in 1997? (Answer: 2)

In [7]:
select count(*) [Number of Titanic Movies]
from imdb.Movies
where Title = 'Titanic'
  and YearReleased <> 1997

(1 row affected)

Total execution time: 00:00:00.024

Number of Titanic Movies
2


How long was the **longest** movie released in _1997_  (the year the epically long _Titanic_ movie was released)?  (360 minutes!)

In [11]:
select MAX(RunningTimeInMinutes) [Longest Movie Time in 1997]
from imdb.Movies
where YearReleased = 1997

(1 row affected)

Total execution time: 00:00:00.032

Longest Movie Time in 1997
360


What are the **smallest**, **average**, and **largest** movie running times that _Leonardo DiCaprio_ has been in?  Name the columns in the resulting table `Shortest`, `Average`, and `Longest`.  (45, 119, and 194 minutes)

In [12]:
with MinLD
as
(
select MIN(RunningTimeInMinutes) [Shortest] from imdb.Movies
JOIN imdb.Principals
  on Movies.TitleID = Principals.TitleID
JOIN imdb.People
  on Principals.PersonID = People.PersonID
where FirstName = 'Leonardo'
  and LastName = 'DiCaprio'
),
AvgLD
as
(
select AVG(RunningTimeInMinutes) [Average] from imdb.Movies
JOIN imdb.Principals
  on Movies.TitleID = Principals.TitleID
JOIN imdb.People
  on Principals.PersonID = People.PersonID
where FirstName = 'Leonardo'
  and LastName = 'DiCaprio'
),
MaxLD
as
(
select MAX(RunningTimeInMinutes) [Longest] from imdb.Movies
JOIN imdb.Principals
  on Movies.TitleID = Principals.TitleID
JOIN imdb.People
  on Principals.PersonID = People.PersonID
where FirstName = 'Leonardo'
  and LastName = 'DiCaprio'
)
SELECT * from MinLD, AvgLD, MaxLD

(1 row affected)

Total execution time: 00:00:00.087

Shortest,Average,Longest
45,119,194


Which movie(s) released in _1997_ was the longest, and who was its **director**? To answer this question

- use the `WITH` statement shown above to allow you to use the query you created to find the movie's length; name this virtual table `Longest1997`
- join `Movies` to `Longest1997` on the `RunningTimeInMinutes` column
- restrict the rows from `Movies` to have a `YearReleased` of _1997_
- join `Movies` to `Principals` and `People` and find the movie's director

In [25]:
with Longest1997
as
(
select Max(RunningTimeInMinutes) LongestRunningTime from imdb.Movies
where YearReleased = 1997
)
select FullName from imdb.Movies
JOIN Longest1997
  on Movies.RunningTimeInMinutes = Longest1997.LongestRunningTime
JOIN imdb.Principals
  on Movies.TitleID = Principals.TitleID
JOIN imdb.People
  on People.PersonID = Principals.PersonID
where Movies.YearReleased = 1997
  and Category = 'Director'


(1 row affected)

Total execution time: 00:00:00.052

FullName
Sue Williams


In [3]:
use [SalesOrdersExample];
select Customers.CustomerID, CustFirstName, CustLastName
from Customers
where not exists (
  select *
  from Orders
  join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
  join Products
  on Products.ProductNumber = Order_Details.ProductNumber
   where Customers.CustomerID = Orders.CustomerID
   and ProductName like '%helmet%'
)
group by Customers.CustomerID, CustFirstName, CustLastName


(3 rows affected)

Total execution time: 00:00:00.055

CustomerID,CustFirstName,CustLastName
1011,Alaina,Hallmark
1028,Jeffrey,Tirekicker
1023,Julia,Schnebly
